In [21]:
from sympy import *
from sympy.parsing.sympy_parser import parse_expr
from sympy.solvers import solve
from sympy.printing.printer import Printer

Printer.set_global_settings(num_columns=2000)

In [22]:
precision = 5

def str_to_exprs(str):
    try:
        result = [expand(N(parse_expr(i.split('=')[0]), precision) - N(
            parse_expr(i.split('=')[1]), precision)) for i in str.split("\n") if i != ""]
        print('Let:')
        pprint(result)
        print()
        return [expand(N(parse_expr(i.split('=')[0]), precision) - N(parse_expr(i.split('=')[1]), precision)) for i in str.split("\n") if i != ""]
    except:
        result = [N(parse_expr(i), precision)
                  for i in str.split("\n") if i != ""]
        print('Let:')
        pprint(result)
        print()
        return [N(parse_expr(i), precision) for i in str.split("\n") if i != ""]


def str_to_dist(str):
    result = {parse_expr(i.split("=")[0]): N(parse_expr(
        i.split("=")[1]), precision) for i in str.split("\n") if i != ""}
    print('Let:')
    pprint(result)
    print()
    return result


def subs_to_group(group, subs):
    result = {key: value.subs(subs) for key, value in group.items()}
    print('Substitution:')
    pprint(result)
    print()
    return result


def subs_to_exprs(exprs, subs):
    result = [i.subs(subs) for i in exprs.items()]
    print('Substitution:')
    pprint(result)
    print()
    return result


def substitute(*args):
    length = len(args)
    if length is 1:
        return args[0]
    if type(args[length - 2]) is dict:
        return substitute(*args[0:-2], subs_to_group(args[length - 2], args[length - 1]))
    elif type(args[length - 2]) is list:
        return substitute(*args[0:-2], subs_to_exprs(args[length - 2], args[length - 1]))


def m_solve_3(exprs, variables, subs):
    A, y = linear_eq_to_matrix(exprs, variables)
    print("original:")
    pprint(Eq(UnevaluatedExpr(A) *
              UnevaluatedExpr(Matrix(variables)), UnevaluatedExpr(y)))
    print('\n\n')

    A = A.subs(subs)
    y = y.subs(subs)
    print("Substituted:")
    pprint(Eq(UnevaluatedExpr(A) *
              UnevaluatedExpr(Matrix(variables)), UnevaluatedExpr(y)))
    print('\n\n')
    
    for i in range(0, A.shape[0]):
        Aii = A[i, i];
        for j in reversed(range(i + 1, A.shape[0])):
            if A[j, i] == 0:
                continue
            print("(Eq" + str(j+1) + ")-(" +
                  str(A[j,i]/A[i,i]) + ")(Eq" + str(i+1) + "):")
            y[j] = y[j] - y[i]*A[j,i]/A[i,i]
            for k in reversed(range(i, A.shape[1])):
                A[j, k] = A[j, k] - A[i, k]*A[j,i]/A[i,i]
            A[j,i] = 0
            pprint((A, y))
            print('\n\n')
    
    ans = solve([expr.subs(subs) for expr in exprs], variables)
    print("Ans:")
    pprint(ans)
    return {k: N(v, precision) for k, v in ans.items()}
    

In [23]:
m_solve_3(str_to_exprs("""
k1*(u2 - u1) - P = 0
k2*(u3-u2) - k1*(u2 - u1) = 0
k3*(u4-u3) - k2*(u3 - u2) = 0
k4*(u5-u4) - k3*(u4 - u3) = 0
k5*(u6-u5) - k4*(u5 - u4) = 0
"""), str_to_exprs("""
u1
u2
u3
u4
u5
"""), substitute(str_to_dist("""
k1 = A1 * _E / l1
k2 = A2 * _E / l2
k3 = A3 * _E / l3
k4 = A4 * _E / l4
k5 = A5 * _E / l5
P = 500
u6 = 0
"""), str_to_dist("""
A1 = 3*(12 + 10.5)/2
A2 = 3*(9 + 10.5)/2
A3 = 3*9
A4 = 3*(9 + 10.5)/2
A5 = 3*(12 + 10.5)/2

l1 = 6
l2 = 6
l3 = 4
l4 = 6
l5 = 6

_E = 3.27e6
""")))

Let:
[-P - k₁⋅u₁ + k₁⋅u₂, k₁⋅u₁ - k₁⋅u₂ - k₂⋅u₂ + k₂⋅u₃, k₂⋅u₂ - k₂⋅u₃ - k₃⋅u₃ + k₃
⋅u₄, k₃⋅u₃ - k₃⋅u₄ - k₄⋅u₄ + k₄⋅u₅, k₄⋅u₄ - k₄⋅u₅ - k₅⋅u₅ + k₅⋅u₆]

Let:
[u₁, u₂, u₃, u₄, u₅]

Let:
⎧              A₁⋅_E      A₂⋅_E      A₃⋅_E      A₄⋅_E      A₅⋅_E       ⎫
⎨P: 500.0, k₁: ─────, k₂: ─────, k₃: ─────, k₄: ─────, k₅: ─────, u₆: 0⎬
⎩                l₁         l₂         l₃         l₄         l₅        ⎭

Let:
{A₁: 33.75, A₂: 29.25, A₃: 27.0, A₄: 29.25, A₅: 33.75, _E: 3.27e+6, l₁: 6.0, l
₂: 6.0, l₃: 4.0, l₄: 6.0, l₅: 6.0}

Substitution:
{P: 500.0, k₁: 1.8394e+7, k₂: 1.5941e+7, k₃: 2.2073e+7, k₄: 1.5941e+7, k₅: 1.8
394e+7, u₆: 0}

original:
⎡-k₁     k₁        0         0         0    ⎤ ⎡u₁⎤   ⎡  P   ⎤
⎢                                           ⎥ ⎢  ⎥   ⎢      ⎥
⎢k₁   -k₁ - k₂     k₂        0         0    ⎥ ⎢u₂⎥   ⎢  0   ⎥
⎢                                           ⎥ ⎢  ⎥   ⎢      ⎥
⎢ 0      k₂     -k₂ - k₃     k₃        0    ⎥⋅⎢u₃⎥ = ⎢  0   ⎥
⎢                                           ⎥ ⎢

{u1: -0.00013975,
 u2: -0.00011257,
 u3: -8.1202e-5,
 u4: -5.8549e-5,
 u5: -2.7183e-5}

In [9]:
A = [36, 33.75, 31.5, 29.25, 27, 27, 27, 29.25, 31.5, 33.75, 36]
for i in range(len(A)-1):
    print(i+1, 1000/(A[i] + A[i+1]))

1 14.336917562724015
2 15.32567049808429
3 16.46090534979424
4 17.77777777777778
5 18.51851851851852
6 18.51851851851852
7 17.77777777777778
8 16.46090534979424
9 15.32567049808429
10 14.336917562724015


In [6]:
3*(-0.25*9+12)

29.25

In [ ]:
100/5()